In [1]:
import pandas as pd
import seaborn as sns
import matplotlib as plt
import re
import numpy as np
from datetime import datetime

In [2]:
#importing the excel spreadsheet containing my police data as a csv. Also using .head() and .info() to take a look at all the rows in the dataset to prepare for cleaning. 
df = pd.read_csv(r"C:\Users\kesha\Documents\nss_da10\projects\police-response-capstone\data\Metro_Nashville_Police_Department_Incidents_20240217.csv")

In [4]:
df.head()

,Primary Key,Incident Number,Report Type,Report Type Description,Incident Status Code,Incident Status Description,Investigation Status,Incident Occurred,Incident Reported,Incident Location,...,Weapon Description,Victim Number,Domestic Related,Victim Type,Victim Description,Victim Gender,Victim Race,Victim Ethnicity,Victim County Resident,Mapped Location
0,20190496213_11,20190496213,D,DISPATCHED,U,UNFOUNDED,Closed,7/1/2019 17:31,7/1/2019 18:17,2410 2410,...,NONE,1,True,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,RESIDENT,POINT (-86.696 36.218)
1,20180290646_11,20180290646,D,DISPATCHED,A,CLEARED BY ARREST,Closed,3/31/2018 12:21,3/31/2018 19:41,5824 5824,...,NONE,1,False,B,BUSINESS,NaN,NaN,NaN,NaN,POINT (-86.713 36.042)
2,20200243837_11,20200243837,D,DISPATCHED,U,UNFOUNDED,Closed,4/6/2020 11:15,4/6/2020 13:11,3144 3144,...,NONE,1,False,G,GOVERNMENT,NaN,NaN,NaN,NaN,POINT (-86.759 36.232)
3,20170471747_11,20170471747,D,DISPATCHED,U,UNFOUNDED,Closed,5/29/2017 2:33,5/29/2017 4:27,1714 1714,...,NONE,1,False,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,RESIDENT,POINT (-86.907 36.086)
4,20171099391_21,20171099391,D,DISPATCHED,A,CLEARED BY ARREST,Closed,12/16/2017 18:30,12/16/2017 22:44,1101 1101,...,PERSONAL (HANDS),1,True,I,INDIVIDUAL (18 AND OVER),F,B,Non-Hispanic,RESIDENT,POINT (-86.586 36.178)


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 865175 entries, 0 to 865174
Data columns (total 31 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Primary Key                  865175 non-null  object 
 1   Incident Number              865175 non-null  int64  
 2   Report Type                  865089 non-null  object 
 3   Report Type Description      805823 non-null  object 
 4   Incident Status Code         865155 non-null  object 
 5   Incident Status Description  865145 non-null  object 
 6   Investigation Status         865175 non-null  object 
 7   Incident Occurred            865175 non-null  object 
 8   Incident Reported            865175 non-null  object 
 9   Incident Location            859872 non-null  object 
 10  Latitude                     848238 non-null  float64
 11  Longitude                    848238 non-null  float64
 12  ZIP Code                     454148 non-null  float64
 13 

In [6]:
#pulled in the only columns I needed from the entire police inident sheet
officers = df[['Report Type Description','Investigation Status','Incident Occurred','Incident Reported','ZIP Code']]
officers


,Report Type Description,Investigation Status,Incident Occurred,Incident Reported,ZIP Code
0,DISPATCHED,Closed,7/1/2019 17:31,7/1/2019 18:17,37214.0
1,DISPATCHED,Closed,3/31/2018 12:21,3/31/2018 19:41,37013.0
2,DISPATCHED,Closed,4/6/2020 11:15,4/6/2020 13:11,37207.0
3,DISPATCHED,Closed,5/29/2017 2:33,5/29/2017 4:27,37205.0
4,DISPATCHED,Closed,12/16/2017 18:30,12/16/2017 22:44,37076.0
...,...,...,...,...,...
865170,DISPATCHED,Closed,2/15/2024 8:00,2/15/2024 10:12,37211.0
865171,DISPATCHED,Open,2/13/2024 22:30,2/14/2024 12:32,NaN
865172,DISPATCHED,Open,2/16/2024 10:00,2/16/2024 15:22,NaN
865173,DISPATCHED,Open,2/15/2024 5:00,2/16/2024 0:03,NaN


In [7]:
mnpd_officers = officers.loc[officers['Report Type Description'] == "DISPATCHED"]
mnpd_officers


,Report Type Description,Investigation Status,Incident Occurred,Incident Reported,ZIP Code
0,DISPATCHED,Closed,7/1/2019 17:31,7/1/2019 18:17,37214.0
1,DISPATCHED,Closed,3/31/2018 12:21,3/31/2018 19:41,37013.0
2,DISPATCHED,Closed,4/6/2020 11:15,4/6/2020 13:11,37207.0
3,DISPATCHED,Closed,5/29/2017 2:33,5/29/2017 4:27,37205.0
4,DISPATCHED,Closed,12/16/2017 18:30,12/16/2017 22:44,37076.0
...,...,...,...,...,...
865170,DISPATCHED,Closed,2/15/2024 8:00,2/15/2024 10:12,37211.0
865171,DISPATCHED,Open,2/13/2024 22:30,2/14/2024 12:32,NaN
865172,DISPATCHED,Open,2/16/2024 10:00,2/16/2024 15:22,NaN
865173,DISPATCHED,Open,2/15/2024 5:00,2/16/2024 0:03,NaN


In [8]:
npd_officers = mnpd_officers.loc[officers['Investigation Status'] == "Closed"]
npd_officers

,Report Type Description,Investigation Status,Incident Occurred,Incident Reported,ZIP Code
0,DISPATCHED,Closed,7/1/2019 17:31,7/1/2019 18:17,37214.0
1,DISPATCHED,Closed,3/31/2018 12:21,3/31/2018 19:41,37013.0
2,DISPATCHED,Closed,4/6/2020 11:15,4/6/2020 13:11,37207.0
3,DISPATCHED,Closed,5/29/2017 2:33,5/29/2017 4:27,37205.0
4,DISPATCHED,Closed,12/16/2017 18:30,12/16/2017 22:44,37076.0
...,...,...,...,...,...
865161,DISPATCHED,Closed,1/31/2024 19:00,1/31/2024 23:07,37115.0
865164,DISPATCHED,Closed,2/9/2024 10:17,2/9/2024 14:09,37013.0
865167,DISPATCHED,Closed,11/7/2023 15:58,11/7/2023 18:34,37203.0
865168,DISPATCHED,Closed,2/11/2024 14:00,2/11/2024 16:15,37204.0


In [9]:
npd_officers.info()

<class 'pandas.core.frame.DataFrame'>
Index: 362799 entries, 0 to 865170
Data columns (total 5 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Report Type Description  362799 non-null  object 
 1   Investigation Status     362799 non-null  object 
 2   Incident Occurred        362799 non-null  object 
 3   Incident Reported        362799 non-null  object 
 4   ZIP Code                 362045 non-null  float64
dtypes: float64(1), object(4)
memory usage: 16.6+ MB


In [33]:
npd_officers['Incident Occurred'] = npd_officers['Incident Occurred'].astype("string")
npd_officers['Incident Reported'] = npd_officers['Incident Reported'].astype("string")

C:\Users\kesha\AppData\Local\Temp\ipykernel_14988\1760736431.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  npd_officers['Incident Occurred'] = npd_officers['Incident Occurred'].astype("string")
C:\Users\kesha\AppData\Local\Temp\ipykernel_14988\1760736431.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  npd_officers['Incident Reported'] = npd_officers['Incident Reported'].astype("string")


In [34]:
npd_officers.info()

<class 'pandas.core.frame.DataFrame'>
Index: 362799 entries, 0 to 865170
Data columns (total 6 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   Report Type Description     362799 non-null  object        
 1   Investigation Status        362799 non-null  object        
 2   Incident Occurred           362799 non-null  string        
 3   Incident Reported           362799 non-null  string        
 4   ZIP Code                    362045 non-null  float64       
 5   Incident Occurred Datetime  362799 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(2), string(2)
memory usage: 19.4+ MB


In [35]:
npd_officers_clean = npd_officers_clean.reset_index()

In [36]:
npd_officers_clean

,level_0,index,Report Type Description,Investigation Status,Incident Occurred,Incident Reported,ZIP Code,Incident Occurred Datetime,occurred_year,occurred_hour_minutes
0,0,0,DISPATCHED,Closed,7/1/2019 17:31,7/1/2019 18:17,37214.0,2019-07-01 17:31:00,2019,17:31
1,1,1,DISPATCHED,Closed,3/31/2018 12:21,3/31/2018 19:41,37013.0,2018-03-31 12:21:00,2018,12:21
2,2,2,DISPATCHED,Closed,4/6/2020 11:15,4/6/2020 13:11,37207.0,2020-04-06 11:15:00,2020,11:15
3,3,3,DISPATCHED,Closed,5/29/2017 2:33,5/29/2017 4:27,37205.0,2017-05-29 02:33:00,2017,02:33
4,4,4,DISPATCHED,Closed,12/16/2017 18:30,12/16/2017 22:44,37076.0,2017-12-16 18:30:00,2017,18:30
...,...,...,...,...,...,...,...,...,...,...
362793,362794,865161,DISPATCHED,Closed,1/31/2024 19:00,1/31/2024 23:07,37115.0,2024-01-31 19:00:00,2024,19:00
362794,362795,865164,DISPATCHED,Closed,2/9/2024 10:17,2/9/2024 14:09,37013.0,2024-02-09 10:17:00,2024,10:17
362795,362796,865167,DISPATCHED,Closed,11/7/2023 15:58,11/7/2023 18:34,37203.0,2023-11-07 15:58:00,2023,15:58
362796,362797,865168,DISPATCHED,Closed,2/11/2024 14:00,2/11/2024 16:15,37204.0,2024-02-11 14:00:00,2024,14:00


In [38]:
npd_officers_clean = npd_officers_clean.drop(295058)

KeyError: '[295058] not found in axis'

In [39]:
npd_officers_clean['Incident Reported Datetime'] = pd.to_datetime(npd_officers_clean['Incident Reported'])

In [40]:
npd_officers_clean

,level_0,index,Report Type Description,Investigation Status,Incident Occurred,Incident Reported,ZIP Code,Incident Occurred Datetime,occurred_year,occurred_hour_minutes,Incident Reported Datetime
0,0,0,DISPATCHED,Closed,7/1/2019 17:31,7/1/2019 18:17,37214.0,2019-07-01 17:31:00,2019,17:31,2019-07-01 18:17:00
1,1,1,DISPATCHED,Closed,3/31/2018 12:21,3/31/2018 19:41,37013.0,2018-03-31 12:21:00,2018,12:21,2018-03-31 19:41:00
2,2,2,DISPATCHED,Closed,4/6/2020 11:15,4/6/2020 13:11,37207.0,2020-04-06 11:15:00,2020,11:15,2020-04-06 13:11:00
3,3,3,DISPATCHED,Closed,5/29/2017 2:33,5/29/2017 4:27,37205.0,2017-05-29 02:33:00,2017,02:33,2017-05-29 04:27:00
4,4,4,DISPATCHED,Closed,12/16/2017 18:30,12/16/2017 22:44,37076.0,2017-12-16 18:30:00,2017,18:30,2017-12-16 22:44:00
...,...,...,...,...,...,...,...,...,...,...,...
362793,362794,865161,DISPATCHED,Closed,1/31/2024 19:00,1/31/2024 23:07,37115.0,2024-01-31 19:00:00,2024,19:00,2024-01-31 23:07:00
362794,362795,865164,DISPATCHED,Closed,2/9/2024 10:17,2/9/2024 14:09,37013.0,2024-02-09 10:17:00,2024,10:17,2024-02-09 14:09:00
362795,362796,865167,DISPATCHED,Closed,11/7/2023 15:58,11/7/2023 18:34,37203.0,2023-11-07 15:58:00,2023,15:58,2023-11-07 18:34:00
362796,362797,865168,DISPATCHED,Closed,2/11/2024 14:00,2/11/2024 16:15,37204.0,2024-02-11 14:00:00,2024,14:00,2024-02-11 16:15:00


In [41]:
npd_officers_clean['occurred_year'] = npd_officers_clean['Incident Occurred Datetime'].dt.year
npd_officers_clean['occurred_hour_minutes'] = npd_officers_clean['Incident Occurred Datetime'].dt.strftime('%H:%M')

In [42]:
npd_officers_clean['reported_year'] = npd_officers_clean['Incident Reported Datetime'].dt.year
npd_officers_clean['reported_hour_minutes'] = npd_officers_clean['Incident Reported Datetime'].dt.strftime('%H:%M')

In [43]:
npd_officers_clean

,level_0,index,Report Type Description,Investigation Status,Incident Occurred,Incident Reported,ZIP Code,Incident Occurred Datetime,occurred_year,occurred_hour_minutes,Incident Reported Datetime,reported_year,reported_hour_minutes
0,0,0,DISPATCHED,Closed,7/1/2019 17:31,7/1/2019 18:17,37214.0,2019-07-01 17:31:00,2019,17:31,2019-07-01 18:17:00,2019,18:17
1,1,1,DISPATCHED,Closed,3/31/2018 12:21,3/31/2018 19:41,37013.0,2018-03-31 12:21:00,2018,12:21,2018-03-31 19:41:00,2018,19:41
2,2,2,DISPATCHED,Closed,4/6/2020 11:15,4/6/2020 13:11,37207.0,2020-04-06 11:15:00,2020,11:15,2020-04-06 13:11:00,2020,13:11
3,3,3,DISPATCHED,Closed,5/29/2017 2:33,5/29/2017 4:27,37205.0,2017-05-29 02:33:00,2017,02:33,2017-05-29 04:27:00,2017,04:27
4,4,4,DISPATCHED,Closed,12/16/2017 18:30,12/16/2017 22:44,37076.0,2017-12-16 18:30:00,2017,18:30,2017-12-16 22:44:00,2017,22:44
...,...,...,...,...,...,...,...,...,...,...,...,...,...
362793,362794,865161,DISPATCHED,Closed,1/31/2024 19:00,1/31/2024 23:07,37115.0,2024-01-31 19:00:00,2024,19:00,2024-01-31 23:07:00,2024,23:07
362794,362795,865164,DISPATCHED,Closed,2/9/2024 10:17,2/9/2024 14:09,37013.0,2024-02-09 10:17:00,2024,10:17,2024-02-09 14:09:00,2024,14:09
362795,362796,865167,DISPATCHED,Closed,11/7/2023 15:58,11/7/2023 18:34,37203.0,2023-11-07 15:58:00,2023,15:58,2023-11-07 18:34:00,2023,18:34
362796,362797,865168,DISPATCHED,Closed,2/11/2024 14:00,2/11/2024 16:15,37204.0,2024-02-11 14:00:00,2024,14:00,2024-02-11 16:15:00,2024,16:15


In [44]:
npd_officers_clean['Occurred Time'] = pd.to_datetime(npd_officers_clean['occurred_hour_minutes'])

C:\Users\kesha\AppData\Local\Temp\ipykernel_14988\2071434028.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  npd_officers_clean['Occurred Time'] = pd.to_datetime(npd_officers_clean['occurred_hour_minutes'])


In [45]:
npd_officers_clean['Reported Time'] = pd.to_datetime(npd_officers_clean['reported_hour_minutes'])

C:\Users\kesha\AppData\Local\Temp\ipykernel_14988\1434245554.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  npd_officers_clean['Reported Time'] = pd.to_datetime(npd_officers_clean['reported_hour_minutes'])


In [56]:
npd_officers_clean['Time Taken'] = npd_officers_clean['Occurred Time'] - npd_officers_clean['Reported Time']

In [57]:
npd_officers_clean

,level_0,index,Report Type Description,Investigation Status,Incident Occurred,Incident Reported,ZIP Code,Incident Occurred Datetime,occurred_year,occurred_hour_minutes,Incident Reported Datetime,reported_year,reported_hour_minutes,Occurred Time,Reported Time,Time Taken,Minutes Taken
0,0,0,DISPATCHED,Closed,7/1/2019 17:31,7/1/2019 18:17,37214.0,2019-07-01 17:31:00,2019,17:31,2019-07-01 18:17:00,2019,18:17,2024-02-26 17:31:00,2024-02-26 18:17:00,-1 days +23:14:00,-0.031944
1,1,1,DISPATCHED,Closed,3/31/2018 12:21,3/31/2018 19:41,37013.0,2018-03-31 12:21:00,2018,12:21,2018-03-31 19:41:00,2018,19:41,2024-02-26 12:21:00,2024-02-26 19:41:00,-1 days +16:40:00,-0.305556
2,2,2,DISPATCHED,Closed,4/6/2020 11:15,4/6/2020 13:11,37207.0,2020-04-06 11:15:00,2020,11:15,2020-04-06 13:11:00,2020,13:11,2024-02-26 11:15:00,2024-02-26 13:11:00,-1 days +22:04:00,-0.080556
3,3,3,DISPATCHED,Closed,5/29/2017 2:33,5/29/2017 4:27,37205.0,2017-05-29 02:33:00,2017,02:33,2017-05-29 04:27:00,2017,04:27,2024-02-26 02:33:00,2024-02-26 04:27:00,-1 days +22:06:00,-0.079167
4,4,4,DISPATCHED,Closed,12/16/2017 18:30,12/16/2017 22:44,37076.0,2017-12-16 18:30:00,2017,18:30,2017-12-16 22:44:00,2017,22:44,2024-02-26 18:30:00,2024-02-26 22:44:00,-1 days +19:46:00,-0.176389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362793,362794,865161,DISPATCHED,Closed,1/31/2024 19:00,1/31/2024 23:07,37115.0,2024-01-31 19:00:00,2024,19:00,2024-01-31 23:07:00,2024,23:07,2024-02-26 19:00:00,2024-02-26 23:07:00,-1 days +19:53:00,-0.171528
362794,362795,865164,DISPATCHED,Closed,2/9/2024 10:17,2/9/2024 14:09,37013.0,2024-02-09 10:17:00,2024,10:17,2024-02-09 14:09:00,2024,14:09,2024-02-26 10:17:00,2024-02-26 14:09:00,-1 days +20:08:00,-0.161111
362795,362796,865167,DISPATCHED,Closed,11/7/2023 15:58,11/7/2023 18:34,37203.0,2023-11-07 15:58:00,2023,15:58,2023-11-07 18:34:00,2023,18:34,2024-02-26 15:58:00,2024-02-26 18:34:00,-1 days +21:24:00,-0.108333
362796,362797,865168,DISPATCHED,Closed,2/11/2024 14:00,2/11/2024 16:15,37204.0,2024-02-11 14:00:00,2024,14:00,2024-02-11 16:15:00,2024,16:15,2024-02-26 14:00:00,2024-02-26 16:15:00,-1 days +21:45:00,-0.093750


In [48]:
#filtered the year for 2023 to match the year of the data in my other notebooks. Seen there were approximately 53,000 dispatched, and closed investigations for Nashville in 2023
npd_officers_clean.loc[npd_officers_clean['reported_year'] == 2023]

,level_0,index,Report Type Description,Investigation Status,Incident Occurred,Incident Reported,ZIP Code,Incident Occurred Datetime,occurred_year,occurred_hour_minutes,Incident Reported Datetime,reported_year,reported_hour_minutes,Occurred Time,Reported Time,Time Taken
83,83,145,DISPATCHED,Closed,6/1/2023 18:30,6/1/2023 19:40,37072.0,2023-06-01 18:30:00,2023,18:30,2023-06-01 19:40:00,2023,19:40,2024-02-26 18:30:00,2024-02-26 19:40:00,-1 days +22:50:00
1523,1523,3540,DISPATCHED,Closed,5/30/2023 17:00,6/2/2023 16:51,37217.0,2023-05-30 17:00:00,2023,17:00,2023-06-02 16:51:00,2023,16:51,2024-02-26 17:00:00,2024-02-26 16:51:00,0 days 00:09:00
2134,2134,4867,DISPATCHED,Closed,6/3/2023 5:30,6/3/2023 7:56,37219.0,2023-06-03 05:30:00,2023,05:30,2023-06-03 07:56:00,2023,07:56,2024-02-26 05:30:00,2024-02-26 07:56:00,-1 days +21:34:00
2188,2188,5020,DISPATCHED,Closed,6/4/2023 15:58,6/4/2023 15:58,37216.0,2023-06-04 15:58:00,2023,15:58,2023-06-04 15:58:00,2023,15:58,2024-02-26 15:58:00,2024-02-26 15:58:00,0 days 00:00:00
2270,2270,5226,DISPATCHED,Closed,6/1/2023 10:06,6/2/2023 10:06,37216.0,2023-06-01 10:06:00,2023,10:06,2023-06-02 10:06:00,2023,10:06,2024-02-26 10:06:00,2024-02-26 10:06:00,0 days 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362784,362785,865137,DISPATCHED,Closed,9/24/2023 17:00,9/28/2023 18:10,37013.0,2023-09-24 17:00:00,2023,17:00,2023-09-28 18:10:00,2023,18:10,2024-02-26 17:00:00,2024-02-26 18:10:00,-1 days +22:50:00
362788,362789,865148,DISPATCHED,Closed,8/12/2023 13:00,8/12/2023 14:15,37214.0,2023-08-12 13:00:00,2023,13:00,2023-08-12 14:15:00,2023,14:15,2024-02-26 13:00:00,2024-02-26 14:15:00,-1 days +22:45:00
362790,362791,865156,DISPATCHED,Closed,9/14/2023 19:00,9/14/2023 21:28,37208.0,2023-09-14 19:00:00,2023,19:00,2023-09-14 21:28:00,2023,21:28,2024-02-26 19:00:00,2024-02-26 21:28:00,-1 days +21:32:00
362792,362793,865158,DISPATCHED,Closed,11/2/2023 7:30,11/2/2023 9:05,37203.0,2023-11-02 07:30:00,2023,07:30,2023-11-02 09:05:00,2023,09:05,2024-02-26 07:30:00,2024-02-26 09:05:00,-1 days +22:25:00


In [49]:
npd_officers_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 362797 entries, 0 to 362797
Data columns (total 16 columns):
 #   Column                      Non-Null Count   Dtype          
---  ------                      --------------   -----          
 0   level_0                     362797 non-null  int64          
 1   index                       362797 non-null  int64          
 2   Report Type Description     362797 non-null  object         
 3   Investigation Status        362797 non-null  object         
 4   Incident Occurred           362797 non-null  string         
 5   Incident Reported           362797 non-null  string         
 6   ZIP Code                    362043 non-null  float64        
 7   Incident Occurred Datetime  362797 non-null  datetime64[ns] 
 8   occurred_year               362797 non-null  int32          
 9   occurred_hour_minutes       362797 non-null  object         
 10  Incident Reported Datetime  362797 non-null  datetime64[ns] 
 11  reported_year               362

In [63]:
npd_officers_clean['Minutes Taken'] = npd_officers_clean['Time Taken'] / pd.to_timedelta(1, unit='minute')


In [64]:
npd_officers_clean

,level_0,index,Report Type Description,Investigation Status,Incident Occurred,Incident Reported,ZIP Code,Incident Occurred Datetime,occurred_year,occurred_hour_minutes,Incident Reported Datetime,reported_year,reported_hour_minutes,Occurred Time,Reported Time,Time Taken,Minutes Taken
0,0,0,DISPATCHED,Closed,7/1/2019 17:31,7/1/2019 18:17,37214.0,2019-07-01 17:31:00,2019,17:31,2019-07-01 18:17:00,2019,18:17,2024-02-26 17:31:00,2024-02-26 18:17:00,-1 days +23:14:00,-46.0
1,1,1,DISPATCHED,Closed,3/31/2018 12:21,3/31/2018 19:41,37013.0,2018-03-31 12:21:00,2018,12:21,2018-03-31 19:41:00,2018,19:41,2024-02-26 12:21:00,2024-02-26 19:41:00,-1 days +16:40:00,-440.0
2,2,2,DISPATCHED,Closed,4/6/2020 11:15,4/6/2020 13:11,37207.0,2020-04-06 11:15:00,2020,11:15,2020-04-06 13:11:00,2020,13:11,2024-02-26 11:15:00,2024-02-26 13:11:00,-1 days +22:04:00,-116.0
3,3,3,DISPATCHED,Closed,5/29/2017 2:33,5/29/2017 4:27,37205.0,2017-05-29 02:33:00,2017,02:33,2017-05-29 04:27:00,2017,04:27,2024-02-26 02:33:00,2024-02-26 04:27:00,-1 days +22:06:00,-114.0
4,4,4,DISPATCHED,Closed,12/16/2017 18:30,12/16/2017 22:44,37076.0,2017-12-16 18:30:00,2017,18:30,2017-12-16 22:44:00,2017,22:44,2024-02-26 18:30:00,2024-02-26 22:44:00,-1 days +19:46:00,-254.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362793,362794,865161,DISPATCHED,Closed,1/31/2024 19:00,1/31/2024 23:07,37115.0,2024-01-31 19:00:00,2024,19:00,2024-01-31 23:07:00,2024,23:07,2024-02-26 19:00:00,2024-02-26 23:07:00,-1 days +19:53:00,-247.0
362794,362795,865164,DISPATCHED,Closed,2/9/2024 10:17,2/9/2024 14:09,37013.0,2024-02-09 10:17:00,2024,10:17,2024-02-09 14:09:00,2024,14:09,2024-02-26 10:17:00,2024-02-26 14:09:00,-1 days +20:08:00,-232.0
362795,362796,865167,DISPATCHED,Closed,11/7/2023 15:58,11/7/2023 18:34,37203.0,2023-11-07 15:58:00,2023,15:58,2023-11-07 18:34:00,2023,18:34,2024-02-26 15:58:00,2024-02-26 18:34:00,-1 days +21:24:00,-156.0
362796,362797,865168,DISPATCHED,Closed,2/11/2024 14:00,2/11/2024 16:15,37204.0,2024-02-11 14:00:00,2024,14:00,2024-02-11 16:15:00,2024,16:15,2024-02-26 14:00:00,2024-02-26 16:15:00,-1 days +21:45:00,-135.0


In [65]:
#filtered the year for 2023 to match the year of the data in my other notebooks. Seen there were approximately 53,000 dispatched, and closed investigations for Nashville in 2023
npd_officers_clean.loc[npd_officers_clean['reported_year'] == 2023]

,level_0,index,Report Type Description,Investigation Status,Incident Occurred,Incident Reported,ZIP Code,Incident Occurred Datetime,occurred_year,occurred_hour_minutes,Incident Reported Datetime,reported_year,reported_hour_minutes,Occurred Time,Reported Time,Time Taken,Minutes Taken
83,83,145,DISPATCHED,Closed,6/1/2023 18:30,6/1/2023 19:40,37072.0,2023-06-01 18:30:00,2023,18:30,2023-06-01 19:40:00,2023,19:40,2024-02-26 18:30:00,2024-02-26 19:40:00,-1 days +22:50:00,-70.0
1523,1523,3540,DISPATCHED,Closed,5/30/2023 17:00,6/2/2023 16:51,37217.0,2023-05-30 17:00:00,2023,17:00,2023-06-02 16:51:00,2023,16:51,2024-02-26 17:00:00,2024-02-26 16:51:00,0 days 00:09:00,9.0
2134,2134,4867,DISPATCHED,Closed,6/3/2023 5:30,6/3/2023 7:56,37219.0,2023-06-03 05:30:00,2023,05:30,2023-06-03 07:56:00,2023,07:56,2024-02-26 05:30:00,2024-02-26 07:56:00,-1 days +21:34:00,-146.0
2188,2188,5020,DISPATCHED,Closed,6/4/2023 15:58,6/4/2023 15:58,37216.0,2023-06-04 15:58:00,2023,15:58,2023-06-04 15:58:00,2023,15:58,2024-02-26 15:58:00,2024-02-26 15:58:00,0 days 00:00:00,0.0
2270,2270,5226,DISPATCHED,Closed,6/1/2023 10:06,6/2/2023 10:06,37216.0,2023-06-01 10:06:00,2023,10:06,2023-06-02 10:06:00,2023,10:06,2024-02-26 10:06:00,2024-02-26 10:06:00,0 days 00:00:00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362784,362785,865137,DISPATCHED,Closed,9/24/2023 17:00,9/28/2023 18:10,37013.0,2023-09-24 17:00:00,2023,17:00,2023-09-28 18:10:00,2023,18:10,2024-02-26 17:00:00,2024-02-26 18:10:00,-1 days +22:50:00,-70.0
362788,362789,865148,DISPATCHED,Closed,8/12/2023 13:00,8/12/2023 14:15,37214.0,2023-08-12 13:00:00,2023,13:00,2023-08-12 14:15:00,2023,14:15,2024-02-26 13:00:00,2024-02-26 14:15:00,-1 days +22:45:00,-75.0
362790,362791,865156,DISPATCHED,Closed,9/14/2023 19:00,9/14/2023 21:28,37208.0,2023-09-14 19:00:00,2023,19:00,2023-09-14 21:28:00,2023,21:28,2024-02-26 19:00:00,2024-02-26 21:28:00,-1 days +21:32:00,-148.0
362792,362793,865158,DISPATCHED,Closed,11/2/2023 7:30,11/2/2023 9:05,37203.0,2023-11-02 07:30:00,2023,07:30,2023-11-02 09:05:00,2023,09:05,2024-02-26 07:30:00,2024-02-26 09:05:00,-1 days +22:25:00,-95.0


In [83]:
npd_officers_cleaner = npd_officers_clean.drop(columns=['Report Type Description', 'Investigation Status'])

In [85]:
#This is a list of zipcodes in the data set. Some of these zip codes will be ignored because they are not in Nashville, are typos, or they are not formally inhabited. 
npd_officers_cleaner.groupby(by='ZIP Code')['Minutes Taken'].mean()

ZIP Code
3701.0     852.000000
17202.0    558.000000
19153.0    445.000000
27216.0     63.000000
27707.0    299.000000
29501.0    355.000000
30139.0    519.000000
30318.0    765.000000
37013.0    263.830041
37015.0    286.289474
37027.0    269.834812
37028.0     29.000000
37043.0    297.000000
37049.0     38.000000
37064.0    536.666667
37066.0    502.000000
37067.0    261.000000
37072.0    271.256136
37075.0    164.333333
37076.0    250.414956
37078.0     66.000000
37080.0    253.916176
37082.0      0.000000
37086.0    221.526316
37087.0    238.666667
37090.0    259.000000
37115.0    231.688187
37122.0    332.326531
37129.0    340.000000
37130.0    336.000000
37135.0    278.817757
37138.0    265.322358
37143.0    426.500000
37167.0    261.333333
37179.0    599.000000
37185.0      0.000000
37189.0    236.937500
37201.0    295.149963
37203.0    245.910447
37204.0    216.830341
37205.0    266.056473
37206.0    213.800739
37207.0    223.639225
37208.0    216.772736
37209.0    228.433625
3